# Insert FrameNet into the Knowledge Graph

Inserts framenet into the knowledge graph
Takes a little while to run

In [ ]:
from nltk.corpus import framenet as fn

In [ ]:
frames_data = []
for frame in fn.frames():
    lexeme_units = list(frame['lexUnit'].keys())
    fe = list(frame["FE"].keys())
    for relation in frame['frameRelations']:
        if relation['type']['name'] == "Inheritance" and relation['Child'] == frame:
            # If it's an inheritance and the child is the frame in question
            lexeme_units += list(relation['Parent']['lexUnit'].keys())
            fe += list(relation['Parent']['FE'].keys())
    frame_data = {
        'name': frame['name'],
        'lexeme_units': lexeme_units,
        'fe': fe
    }
    frames_data.append(frame_data)

In [ ]:
from py2neo import Graph
import os

host = os.environ.get('REMOTE_NEO4J_URL')
user = os.environ.get('NEO4J_USER')
password = os.environ.get('NEO4J_PASSWORD')
graph = Graph(host=host, user='neo4j', password = password, secure=True)

cypher = ""

for index, frame in enumerate(frames_data[447 + 529:]):
    print(f"{index} of {len(frames_data)}")
    graph.run("CREATE (f:Frame {name: '" + frame['name'] + "'})")
    for lexeme_unit in list(set(frame['lexeme_units'])):
        split_lexeme = lexeme_unit.split(".")
        if split_lexeme[1] == "v":
            word = split_lexeme[0].replace("'", "")
            graph.run("MATCH (f:Frame {name: '" + frame['name'] + "'}) \
            MERGE (w:Verb { word: '" + word + "'}) \
            MERGE (w)<-[:HAS_LEXEME_UNIT]-(f)")
    for fe in frame['fe']:
        fe = fe.replace("'", "")
        fe = fe.lower()
        if len(graph.run("MATCH (e:Entity {name: '" + fe + "'}) RETURN e").data()) > 0:
            graph.run("MATCH (f:Frame { name: '" + frame['name'] + "'}), \
                (e:Entity { name: '" + fe + "' }) \
                SET e:FrameElement \
                MERGE (e)<-[:HAS_FRAME_ELEMENT]-(f)")
        else:
            graph.run("MATCH (f:Frame {name: '" + frame['name'] + "'}) \
            MERGE (w:FrameElement { type: '" + fe + "'})<-[:HAS_FRAME_ELEMENT]-(f)")